# Comp5349 Assignment 2
### Team member: Jiakun Yu, Zezheng Zhang, Yihong Wang, Yue Yang

# Stage One: Overall statistics

In [1]:
# Import all necessary libraries and setup the environment for matplotlib
from pyspark.sql import SparkSession
from pyspark.ml.feature import Word2Vec
from pyspark.sql.types import *
from pyspark.sql.functions import udf, explode, split
from pyspark.sql import functions as F
import nltk
import string
import numpy as np

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1558489762291_0006,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
spark = SparkSession \
    .builder \
    .appName("Assignment2") \
    .getOrCreate()

revs_data = "s3://amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz"
revs = spark.read.csv(revs_data,header=True, sep='\t').cache()

VBox()

In [3]:
revs.printSchema()

VBox()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)

In [4]:
revs.head()

VBox()

Row(marketplace='US', customer_id='27288431', review_id='R33UPQQUZQEM8', product_id='B005T4ND06', product_parent='400024643', product_title="Yoga for Movement Disorders DVD: Rebuilding Strength, Balance, and Flexibility for Parkinson's Disease and Dystonia", product_category='Video DVD', star_rating='5', helpful_votes='3', total_votes='3', vine='N', verified_purchase='Y', review_headline="This was a gift for my aunt who has Parkinson's ...", review_body="This was a gift for my aunt who has Parkinson's.  While I have not previewed it myself, I also have not gotten any complaints.  My prior experiences with yoga tell me this should be just what the doctor ordered.", review_date='2015-08-31')

#### The Total number of reviews

In [5]:
revs.count()

VBox()

5069140

#### The number of unique users

In [6]:
revs.select('customer_id').distinct().count()

VBox()

2075970

#### The number of unique products

In [7]:
revs.select('product_id').distinct().count()

VBox()

297919

## User-review distribution

In [8]:
customer_review_count = revs.groupby('customer_id').count().cache()

VBox()

#### (The largest number of reviews published by a single user) and (The top 10 users ranked by the number)

In [9]:
customer_review_count.orderBy('count', ascending=False).show(10)

VBox()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   43430756| 3582|
|   18116317| 2987|
|   52287429| 2747|
|   52496677| 2650|
|   51110953| 2624|
|   19792742| 2495|
|   20018062| 2492|
|   50068216| 2379|
|   14539589| 2269|
|   50881246| 2104|
+-----------+-----+
only showing top 10 rows

#### The median number of reviews published by a user

In [10]:
user_count_median = customer_review_count.approxQuantile(col='count', probabilities=[0.5], relativeError=0)
user_count_median

VBox()

[1.0]

## Product-review distribution

In [11]:
product_review_count = revs.groupby('product_id').count().cache()

VBox()

#### (The largest number of reviews written for a single product) and (The top 10 products ranked by the number of reviews)

In [12]:
product_review_count.orderBy('count', ascending=False).show(10)

VBox()

+----------+-----+
|product_id|count|
+----------+-----+
|B00127RAJY| 4969|
|B0000AQS0F| 4967|
|B00G5G7K7O| 4439|
|B0002IQJ96| 4409|
|B00G5G7EXY| 4207|
|B003ZSJ212| 3641|
|B000X9FLKM| 3414|
|B000K8LV1O| 2989|
|B00N1JQ2UO| 2476|
|B00AMR5LZA| 2253|
+----------+-----+
only showing top 10 rows

#### The median number of reviews a product has

In [13]:
product_count_median = product_review_count.approxQuantile(col='count', probabilities=[0.5], relativeError=0)
product_count_median

VBox()

[3.0]

# Stage Two: Filtering Unwanted Data

In [14]:
def sent_TokenizeFunct(x):
    if(not x):
        sentences = ''
    else:
        sentences=x.lower()
        sentences = nltk.sent_tokenize(sentences)
    return len(sentences)

VBox()

In [15]:
slen = udf(sent_TokenizeFunct, IntegerType())

VBox()

In [16]:
revs_with_TokenLength = revs.withColumn("TokenLength", slen("review_body"))
filtered_revs1 = revs_with_TokenLength.filter("TokenLength> 1")
customer_review_count = customer_review_count.filter("count> 1")
product_review_count = product_review_count.filter("count> 3")

VBox()

In [17]:
filtered_revs2 = filtered_revs1.join(product_review_count, 'product_id', 'inner') \
                                    .drop("count")

final = filtered_revs2.join(customer_review_count, 'customer_id', 'inner') \
                                .drop("count").cache()

VBox()

In [18]:
final.count()

VBox()

2704735
----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 50252)
Traceback (most recent call last):
  File "/usr/lib64/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib64/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 266, in handle
    poll(authenticate_and_accum_updates)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 254, in authenticate_and_accum_updates
    rec

In [19]:
#final.show(10)

VBox()

In [20]:
magic_percentile = F.expr('percentile_approx(TokenLength, 0.5)')

top_10_users = final.groupby("customer_id").agg(magic_percentile.alias("med_TokenLength")) \
                .orderBy('med_TokenLength', ascending=False)
top_10_users.show(10)

VBox()

+-----------+---------------+
|customer_id|med_TokenLength|
+-----------+---------------+
|   11449554|            317|
|   18288540|            278|
|   47939538|            273|
|   51622006|            233|
|   24142448|            218|
|   51722321|            189|
|   39452841|            180|
|   24972624|            174|
|   32704480|            170|
|   46401180|            166|
+-----------+---------------+
only showing top 10 rows

In [21]:
top_10_products = final.groupby("product_id").agg(magic_percentile.alias("med_TokenLength")) \
                .orderBy('med_TokenLength', ascending=False)
top_10_products.show(10)

VBox()

+----------+---------------+
|product_id|med_TokenLength|
+----------+---------------+
|B00U0WY2J8|            149|
|B000085EEJ|            136|
|B0000C0YMR|            132|
|B00RNTYI3G|            118|
|B00BGWDYNQ|            102|
|B002LMOCJA|             98|
|B0040BJH7C|             97|
|B007G17FYK|             89|
|B003H5HGCU|             87|
|B004X6JB98|             85|
+----------+---------------+
only showing top 10 rows

# Stage Three: Similarity analysis with Sentence Em-bedding

## Positive vs. Negative Reviews

In [22]:
product = revs.filter(revs.product_id == 'B00AMR5LZA') \
                    .select('customer_id','review_id','star_rating','review_body').cache()
#product.count()
#product.show()

VBox()

In [23]:
def sent_Tokenize(x):
    if(not x):
        sentences = ''
    else:
        sentences=x.lower()
        sentences = nltk.sent_tokenize(sentences)
    return sentences
s_Tokenize = udf(sent_Tokenize, ArrayType(StringType()))

VBox()

In [24]:
# pos_class = product.filter("star_rating >= 4") \
#                    .withColumn("segmentation", split("review_body", '\. |\! |\? ')).drop("review_body")
# neg_class = product.filter("star_rating <= 2") \
#                    .withColumn("segmentation", split("review_body", '\. |\! |\? ')).drop("review_body")

pos_class = product.filter("star_rating >= 4") \
                   .withColumn("segmentation", s_Tokenize("review_body")).drop("review_body")
neg_class = product.filter("star_rating <= 2") \
                   .withColumn("segmentation", s_Tokenize("review_body")).drop("review_body")

VBox()

In [25]:
seg_pos = pos_class.withColumn("seg",explode("segmentation")).select("seg","review_id")
seg_neg = neg_class.withColumn("seg",explode("segmentation")).select("seg","review_id")

VBox()

In [26]:
import tensorflow as tf
import tensorflow_hub as hub

def review_embed(rev_text_partition):
    module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]
    embed = hub.Module(module_url)
    # mapPartition would supply element inside a partition using generator stype
    # this does not fit tensorflow stype
    rev_text_list = [text for text in rev_text_partition]
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        message_embeddings = session.run(embed(rev_text_list))
    return message_embeddings

VBox()

In [27]:
pos_class_map = seg_pos.rdd.filter(lambda row: len(str(row[0]))>5).cache()
pos_class_rdd = pos_class_map.map(lambda row: str(row[0])).cache()
pos_review_embedding = pos_class_rdd.mapPartitions(review_embed)

VBox()

In [28]:
neg_class_map = seg_neg.rdd.filter(lambda row: len(str(row[0]))>5).cache()
neg_class_rdd = neg_class_map.map(lambda row: str(row[0])).cache()
neg_review_embedding = neg_class_rdd.mapPartitions(review_embed)

VBox()

## Intra-Class Similarity

In [29]:
def Cosine_distance(x):
    a = x[0][0]
    a_id = x[0][1]
    b = x[1][0]
    b_id = x[1][1]

    return ((a_id,b_id), 1 - (a@b)/np.sqrt(((a@a)*(b@b))))

VBox()

##### Positive Class

In [30]:
pos_id = pos_review_embedding.zipWithIndex().cache()
cartesian_rdd = pos_id.cartesian(pos_id).map(Cosine_distance).cache()
#cartesian_rdd.getNumPartitions()
#cartesian_rdd.count()
average_distance = cartesian_rdd \
                    .map(lambda v: (v[0][0], (v[1], 1))) \
                    .reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1])) \
                    .mapValues(lambda v: v[0]/(v[1]-1)).cache()
#average_distance.count()
#average_distance.getNumPartitions()

VBox()

##### Negative Class

In [31]:
neg_id = neg_review_embedding.zipWithIndex().cache()
neg_cartesian_rdd = neg_id.cartesian(neg_id).map(Cosine_distance).cache()
neg_average_distance = neg_cartesian_rdd \
                    .map(lambda v: (v[0][0], (v[1], 1))) \
                    .reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1])) \
                    .mapValues(lambda v: v[0]/(v[1]-1)).cache()

VBox()

## Class Center Sentences

##### Positive Class

In [32]:
Points_num = average_distance.count()
Class_Center = average_distance.takeOrdered(1,lambda x: x[1])
print('The Class_Center and its average dis is:', Class_Center)
closeness = average_distance.map(lambda x: x[1]).sum()/Points_num
print('The Closeness is:', closeness)

VBox()

The Class_Center and its average dis is: [(3072, 0.56196239992007)]
The Closeness is: 0.7074424467782419

In [33]:
Class_Center_index = Class_Center[0][0]
Class_Center_dis = Class_Center[0][1]
ten_closest_neighbours = cartesian_rdd \
                        .filter(lambda x: ((x[0][0]==Class_Center_index) and (x[0][1]!=Class_Center_index))) \
                        .takeOrdered(10,lambda x: x[1])
print('10 closest neighbours and its average distance:')
print(ten_closest_neighbours)

VBox()

10 closest neighbours and its average distance:
[((3072, 5085), 0.06863468885421753), ((3072, 2936), 0.08381921052932739), ((3072, 277), 0.09751492738723755), ((3072, 3547), 0.10153257846832275), ((3072, 237), 0.11305892467498779), ((3072, 1913), 0.11406934261322021), ((3072, 1908), 0.1177402138710022), ((3072, 4321), 0.11930060386657715), ((3072, 4812), 0.12375795841217041), ((3072, 303), 0.12446123361587524)]

In [34]:
sent_map = pos_id.map(lambda x: x[1]).zip(pos_class_map).collectAsMap()
print('Positive Class Center:\n\n',sent_map[Class_Center_index][1]+":",sent_map[Class_Center_index][0])
print('\n\n10 closest neighbours:\n')
for i in range(10):
    print(sent_map[ten_closest_neighbours[i][0][1]][1]+":",sent_map[ten_closest_neighbours[i][0][1]][0])

VBox()

Positive Class Center:

 R3G3KHQ43Y860A: this is the best bible movie ever made!


10 closest neighbours:

RVJRVW37VK5HJ: this is the best bible movie i had ever seen i had seen a lot of bible movies but this is the best
RPHGQC7A7NZHG: one of the best made movie versions of the bible yet.
RRX99ACI7D1QI: of all the bible based films, this was my favorite jesus.
R1P0X6G9JNUSHN: truly a great movie about the bible.
R2A85C5IBC5LVH: one of the most incredible movies out there about the bible!
RFQSUAJDSY6SW: this is one of the best movies that depict the stories of the bible.
R1VV03CKEK08I0: this mini series is one if the best bible movies made.
R1NRPFNQIG56FX: it is one of the best bible stories ever depicted on movie setting if not the best.
R3NFXPKKG26EYT: of all of the biblical movies i've seen, this by far is the best one yet.
RC0FZ9SNACXIG: fantastic movie for bible lovers.

##### Negative Class

In [35]:
Points_num = neg_average_distance.count()
neg_Class_Center = neg_average_distance.takeOrdered(1,lambda x: x[1])
print('The Class_Center and its average dis is:', neg_Class_Center)
closeness = neg_average_distance.map(lambda x: x[1]).sum()/Points_num
print('The Closeness is:', closeness)

VBox()

The Class_Center and its average dis is: [(1088, 0.5773194194771349)]
The Closeness is: 0.7322638106126443

In [36]:
neg_Class_Center_index = neg_Class_Center[0][0]
neg_Class_Center_dis = neg_Class_Center[0][1]
neg_ten_closest_neighbours = neg_cartesian_rdd \
                        .filter(lambda x: ((x[0][0]==neg_Class_Center_index) and (x[0][1]!=neg_Class_Center_index))) \
                        .takeOrdered(10,lambda x: x[1])

print('10 closest neighbours and its average distance:')
print(neg_ten_closest_neighbours)

VBox()

10 closest neighbours and its average distance:
[((1088, 864), 0.17561036348342896), ((1088, 1222), 0.19217908382415771), ((1088, 1221), 0.19503241777420044), ((1088, 424), 0.19555699825286865), ((1088, 186), 0.20735925436019897), ((1088, 1122), 0.21118015050888062), ((1088, 1270), 0.21852785348892212), ((1088, 900), 0.22103667259216309), ((1088, 512), 0.22542214393615723), ((1088, 674), 0.22580265998840332)]

In [37]:
neg_sent_map = neg_id.map(lambda x: x[1]).zip(neg_class_map).collectAsMap()
print('Negative Class Center:\n\n',neg_sent_map[neg_Class_Center_index][1]+":",neg_sent_map[neg_Class_Center_index][0])
print('\n\n10 closest neighbours:\n')
for i in range(10):
    print(neg_sent_map[neg_ten_closest_neighbours[i][0][1]][1]+":",neg_sent_map[neg_ten_closest_neighbours[i][0][1]][0])

VBox()

Negative Class Center:

 R1TQT3UD1NYPEF: the makers of it had a great opportunity to tell the story of the bible.


10 closest neighbours:

R2BOHBJN6W1FFO: i have heard that it was created to introduce people into the bible.
R2TYF4KR0I67D5: good bible story books are really inspired by the bible.
R2TYF4KR0I67D5: they should have done the screenplay off a really accurate bible story book(s) and then went from there, filling in the lose ends(other parts that they wanted to add)with the bible.
R3TPP75ZVB83FE: the bible?
RFHUIBNG40GHX: this is the bible!
R1S1GCV6NJM37Q: the writers should have read the bible before they started butchering the story that they obviously thought they could improve upon.
R3021YB26T5Q0N: this was called the bible, not most of the bible, and the last time i looked the book of revelation was one of the most important books in the bible.
RM3GHVBLVWL5: this was the worst recreation of the bible known to man!
R149PWP0UVY4KY: a simple reading of the bible would show 

# Stage Four: Similarity analysis with Spark supported Feature Extractors

In [38]:
def Cos_dis_word2vec(x):
    a = x[0][0]
    a_id = x[0][1]
    b = x[1][0]
    b_id = x[1][1]

    return ((a_id,b_id), 1 - a.dot(b)/a.norm(2)/b.norm(2))

VBox()

##### Positive Class

In [39]:
pos_class_df = spark.createDataFrame(pos_class_rdd.map(lambda x: [x]), ArrayType(StringType()))
word2Vec = Word2Vec(vectorSize=300, minCount=0, inputCol="value", outputCol="result")
model = word2Vec.fit(pos_class_df)
pos_word2vec = model.transform(pos_class_df).select("result").rdd.map(lambda x:x[0])

pos_word2vec_id = pos_word2vec.zipWithIndex().cache()
pos_word2vec_cartesian = pos_word2vec_id.cartesian(pos_word2vec_id).map(Cos_dis_word2vec).cache()


pos_average_distance = pos_word2vec_cartesian \
                    .map(lambda v: (v[0][0], (v[1], 1))) \
                    .reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1])) \
                    .mapValues(lambda v: v[0]/(v[1]-1)).cache()
print('Word2Vec Closeness is:',(pos_average_distance.map(lambda x:x[1]).sum())/pos_average_distance.count())

pos_Class_Center = pos_average_distance.takeOrdered(1,lambda x: x[1])
pos_Class_Center_index = pos_Class_Center[0][0]
pos_Class_Center_dis = pos_Class_Center[0][1]
pos_ten_closest_neighbours = pos_word2vec_cartesian \
                        .filter(lambda x: ((x[0][0]==pos_Class_Center_index) and (x[0][1]!=pos_Class_Center_index))) \
                        .takeOrdered(10,lambda x: x[1])

pos_sent_map = pos_word2vec_id.map(lambda x: x[1]).zip(pos_class_map).collectAsMap()
print('Positive Class Center:\n\n',pos_sent_map[pos_Class_Center_index][1]+":",pos_sent_map[pos_Class_Center_index][0])
print('\n\n10 closest neighbours:\n')
for i in range(10):
    print(pos_sent_map[pos_ten_closest_neighbours[i][0][1]][1]+":",pos_sent_map[pos_ten_closest_neighbours[i][0][1]][0])

VBox()

Word2Vec Closeness is: 0.9999721382979777
Positive Class Center:

 R1KFSV90H1M9I5: this movie is excellent.


10 closest neighbours:

R1HJUXE8B7WWUK: awesome series if you have not boughten this series then...what are you waiting for?
R2N3XIWU4JID2K: this covers all things of historical fact.
R34GF5Z93B8ELU: it's a great summary that capture your attention.
R1MBWS4LYK4GY6: very good.
R35AWA5YDSETP: very good.
RECGRWZQKIN56: very good.
R16S08JK4654YK: very good.
R2HVMVVIA2Z2PS: in a lot of areas it is not always accurate with scripture, but overall, pretty good.
R1821CPKEEDAMU: a basic for everyone to get their feet wet to be inspired to read the entire bible.
R19LT7UG961QVY: humble and compassionate but authoritative.

##### Negative Class

In [40]:
neg_class_df = spark.createDataFrame(neg_class_rdd.map(lambda x: [x]), ArrayType(StringType()))
# neg_class_df.show()
word2Vec = Word2Vec(vectorSize=300, minCount=0, inputCol="value", outputCol="result")
model = word2Vec.fit(neg_class_df)
neg_word2vec = model.transform(neg_class_df).select("result").rdd.map(lambda x:x[0])

neg_word2vec_id = neg_word2vec.zipWithIndex().cache()
neg_word2vec_cartesian = neg_word2vec_id.cartesian(neg_word2vec_id).map(Cos_dis_word2vec).cache()

VBox()

In [41]:
# hashingTF = HashingTF(1000)
# neg_tf = hashingTF.transform(neg_class_rdd)
# neg_tf.cache()
# neg_idf = IDF(minDocFreq=1).fit(neg_tf)
# neg_tfidf = neg_idf.transform(neg_tf)
# neg_tfidf.take(3)
#neg_tfidf_id = neg_tfidf.zipWithIndex().cache()
#neg_tfidf_cartesian = neg_tfidf_id.cartesian(neg_tfidf_id).map(Cos_dis_tfidf).cache()

VBox()

In [42]:
neg_average_distance = neg_word2vec_cartesian \
                    .map(lambda v: (v[0][0], (v[1], 1))) \
                    .reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1])) \
                    .mapValues(lambda v: v[0]/(v[1]-1)).cache()
print('Word2Vec Closeness is:',(neg_average_distance.map(lambda x:x[1]).sum())/neg_average_distance.count())

neg_Class_Center = neg_average_distance.takeOrdered(1,lambda x: x[1])
neg_Class_Center_index = neg_Class_Center[0][0]
neg_Class_Center_dis = neg_Class_Center[0][1]
neg_ten_closest_neighbours = neg_word2vec_cartesian \
                        .filter(lambda x: ((x[0][0]==neg_Class_Center_index) and (x[0][1]!=neg_Class_Center_index))) \
                        .takeOrdered(10,lambda x: x[1])

neg_sent_map = neg_word2vec_id.map(lambda x: x[1]).zip(neg_class_map).collectAsMap()
print('Negative Class Center:\n\n',neg_sent_map[neg_Class_Center_index][1]+":",neg_sent_map[neg_Class_Center_index][0])
print('\n\n10 closest neighbours:\n')
for i in range(10):
    print(neg_sent_map[neg_ten_closest_neighbours[i][0][1]][1]+":",neg_sent_map[neg_ten_closest_neighbours[i][0][1]][0])

VBox()

Word2Vec Closeness is: 0.9999876696151025
Negative Class Center:

 RQZT7L2KEGVJN: when i saw, the \\"leaking\\" ark, i knew this did not bode well.


10 closest neighbours:

RB52RK9IVGUH6: many details of the individual stories were altered or embellished for, i suppose, dramatic effect, thus the need for a disclaimer at the beginning of each episode, always a warning that the content will be altered beyond the approval of many.
RPNUUWLSPFJTU: horrible!
R2IWBY39860T88: no, now it's reality shows and blood-and-guts vikings and romans and swamp people and heaven knows what's next!
R1OXSBNJ4VDZF3: the writers have not only taken so many liberties with the bible to make this movie &#34;agreeable&#34; to a new audience, but it also changes and/or leaves out so much important aspects of the bible that it shouldn't even be sold in christian book stores.
RRHOQWC1P180Y: in this series, jesus heals a bunch of people and draws a lot of attention to himself.
R20FT0D9JP1CJD: he had hair like lambs 